In [ ]:
import pytest
from flask import Flask
from app import app

# Cette fixture pytest crée un client de test pour notre application Flask.
# Elle configure l'application pour les tests et rend le client disponible pour chaque test.
@pytest.fixture
def client():
    app.config['TESTING'] = True  # Active le mode test, qui capture les exceptions et autres erreurs.
    with app.test_client() as client:  # Utilise le client de test de Flask.
        yield client  # Ce yield permet de réutiliser le client pour plusieurs tests.

# Teste la route '/' pour s'assurer qu'elle renvoie le statut 200 (OK)
# et qu'elle contient le mot "modèle" dans la réponse.
def test_home_route(client):
    response = client.get('/')  # Fait une requête GET sur l'URL de base.
    assert response.status_code == 200  # Vérifie que le statut de la réponse est 200.
    assert "modèle".encode('utf-8') in response.data  # Vérifie que "modèle" est dans les données de la réponse.

# Teste la route '/api/predict' avec un identifiant valide pour s'assurer qu'elle renvoie une prédiction.
def test_predict_route_with_valid_id(client):
    response = client.post('/api/predict', data={'client_id': 123})  # Envoie une requête POST avec un ID valide.
    assert response.status_code == 200  # Confirme que la réponse est 200.
    data = response.get_json()  # Récupère la réponse JSON.
    assert 'prediction' in data  # Vérifie que la réponse contient une "prediction".

# Teste la route '/api/predict' avec un identifiant invalide pour vérifier la gestion des erreurs.
def test_predict_route_with_invalid_id(client):
    response = client.post('/api/predict', data={'client_id': 999})  # Envoie une requête POST avec un ID invalide.
    assert response.status_code == 200  # La réponse devrait toujours être 200, même pour des requêtes non valides.
    data = response.get_json()  # Récupère la réponse JSON.
    assert 'error' in data  # Vérifie que la réponse contient une "erreur".
    assert data['prediction'] is None  # S'assure que "prediction" est nulle.

# Point d'entrée pour lancer les tests.
if __name__ == '__main__':
    pytest.main()
